In [ ]:
# Import Python package
import binhosupernova
from binhosupernova.supernova import Supernova
from binhosupernova.commands.i3c.definitions import *
from binhosupernova.commands.system.definitions import GetUsbStringSubCommand

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``binhosupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [ ]:
binhosupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&39aa8cf6&2&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': 'D6175D813F06E05EA0A6A497E6C926A',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``binhosupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Supernova device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Supernova device is functioning as expected.

In [8]:
# GET USB STRING - Get Manufacturer
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from SUPERNOVA:
{'id': 1, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 12, 'payload': 'MN-Binho LLC'}
>> New message from SUPERNOVA:
{'id': 2, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 18, 'payload': 'PR-Binho Supernova'}
>> New message from SUPERNOVA:
{'id': 3, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 35, 'payload': 'SN-D6175D813F06E05EA0A6A497E6C926A6'}
>> New message from SUPERNOVA:
{'id': 4, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 20, 'payload': 'FW-3.2.0-272-078e740'}
>> New message from SUPERNOVA:
{'id': 5, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 4, 'payload': 'HW-C'}


## I3C Protocol API

The I3C Protocol API methods are described below.

### 1. Configure the Supernova as an I3C Controller

``Supernova.i3cControllerInit(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method initializes the Supernova as an I3C controller.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [9]:
supernova.i3cControllerInit(getId(), I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_250_KHZ, I2cTransferRate._400KHz)

{'module': 1, 'opcode': 0, 'message': 'I3C CONTROLLER INIT request success'}

>> New message from SUPERNOVA:
{'id': 6, 'command': 'I3C CONTROLLER INIT', 'result': 'SUCCESS'}


### 2. Set I3C voltage

``Supernova.setI3cVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the I3C bus in mV, ranging from 800 mV up to 3300 mV. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [800, 3300] mV and allows the value 0 mV to power off the output voltage.

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [10]:
supernova.setI3cVoltage(getId(), 3300)

{'module': 1, 'opcode': 0, 'message': 'SET I3C VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 7, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


### 3. I3C bus initialization

There are two options:
1. Initialize the bus with an **empty target devices table** and scan the I3C bus.
2. Add target to the target device table before initializing the bus.

In this case, the I3C Controller performs an RSTDAA and ENTDAA CCC. If one or more I3C Targets acknowledge the Broadcast Address 0x7E, then the Supernova 
will assign the lower available and valid dynamic address.

Previously, retrieve the target devices table with the method: `Supernova.i3cControllerGetTargetDevicesTable(getId())`

In [11]:
# Get the Target Devices Table from the Supernova. It must be empty before the bus initialization.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 8, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 3, 'table': [{'static_address': 80, 'dynamic_address': 0, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I2C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 107, 'dynamic_address': 10, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_rea

`Supernova.i3cControllerInitBus(id, table)`

- `id: int`: A 2-byte integer that represents the transfer ID.
- `table: dict`: A Python dictionary containing information about the target devices table. Default value: `None`

In [12]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cControllerInitBus(getId())

>> New message from SUPERNOVA:
{'id': 9, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [13]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 10, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 2, 'table': [{'static_address': 0, 'dynamic_address': 8, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 0, 'dynamic_address': 9, 'pid': [7, 112, 16, 67, 16, 0], 'bcr': 6, 'dcr': 239, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_

2. Reset I3C BUS

Let's reset the bus to initialize the bus again, this time sending a non-empty target devices table to the Supernova.

`Supernova.i3cControllerResetBus(id)`

This method requests the Supernova to: 1. issue an RSTDAA CCC, 2. free the already assigned dynamic addresses and 3. clean up the target devices table. 

- `id: int`: A 2-byte integer that represents the transfer ID.

In [14]:
supernova.i3cControllerResetBus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER RESET BUS request success'}

>> New message from SUPERNOVA:
{'id': 11, 'command': 'I3C CONTROLLER RESET BUS', 'result': 'SUCCESS'}


In [15]:
# The target devices table must be empty again after resetting the bus.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

3. Add targets to the target device table before initializing the bus using a Python dictionary

In [16]:
"""
If the static address of a target is known, send it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
"""
LSMD6V_STATIC_ADDRESS = 0x6B        #ST LSM6DSV Static Address

"""
Select the dynamic address to be assigned to the I3C Target.
"""
LSMD6V_DYNAMIC_ADDRESS = 0x0A

"""
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.

In the example below, the SETDASA CCC is used to set the dynamic address set above to the target.
"""
LSMD6V_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.REJECT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.DO_NOT_USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

"""
In the example below, the ENTDAA CCC is used to set the target dynamic address. To identify the target
is mandatory to set the PID, BCR and DCR.

"""
BMI323_DYNAMIC_ADDRESS = 0x0B

BMI323_PID = [0x07, 0x70, 0x10, 0x43, 0x10, 0x00]
BMI323_BCR = 0x06
BMI323_DCR = 0xEF

BMI323_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,
        "CRRequest": ControllerRoleRequest.ACCEPT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.DO_NOT_USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

"""
Add an I2C Device too. In this case, it can be the Sparkfun EEPROM breakout board or the Adafruit I2C FRAM memory.
"""
I2C_STATIC_ADDRESS = 0x50

I2C_CONFIGURATION = {
        "targetType": TargetType.I2C_DEVICE,
        "IBIRequest": TargetInterruptRequest.REJECT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.DO_NOT_USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.DO_NOT_USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

"""
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
"""
table = {0: {'staticAddress': LSMD6V_STATIC_ADDRESS,
        'dynamicAddress': LSMD6V_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'configuration': LSMD6V_CONFIGURATION},
        1: {'staticAddress': 0x00,
        'dynamicAddress': BMI323_DYNAMIC_ADDRESS,
        'bcr': BMI323_BCR,
        'dcr': BMI323_DCR,
        'pid': BMI323_PID,
        'configuration': BMI323_CONFIGURATION},
        2: {'staticAddress': I2C_STATIC_ADDRESS,
        'dynamicAddress': 0x00,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'configuration': I2C_CONFIGURATION}
        }

"""
Initialize the bus using the predefined target devices table.
"""
request_result = supernova.i3cControllerInitBus(getId(), table)

>> New message from SUPERNOVA:
{'id': 12, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 0, 'table': []}
>> New message from SUPERNOVA:
{'id': 13, 'command': 'I3C CONTROLLER INIT BUS', 'result': 'SUCCESS', 'invalid_addresses': []}


In [17]:
# Get the update target devices table.
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 14, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 3, 'table': [{'static_address': 80, 'dynamic_address': 0, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I2C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 107, 'dynamic_address': 10, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_re

### 4. Set/Change target configuration

`Supernova.i3cControllerSetTargetDeviceConfiguration(id, entry)`

- `id: int`: A 2-byte integer that represents the transfer ID.
- `entry: dict`: A Python dictionary containing information about the target whose configuration is set.

In [18]:
NEW_LSMD6V_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,              # Accept IBIs request.
        "CRRequest": ControllerRoleRequest.REJECT_CRR,
        "daaUseSETDASA": SetdasaConfiguration.USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.DO_NOT_USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

supernova.i3cControllerSetTargetDeviceConfiguration(getId(), LSMD6V_DYNAMIC_ADDRESS, NEW_LSMD6V_CONFIGURATION)

{'module': 1,
 'opcode': 0,
 'message': 'I3C SET TARGET DEVICE CONFIG request success'}

>> New message from SUPERNOVA:
{'id': 15, 'command': 'I3C CONTROLLER SET TARGET DEVICE CONFIG', 'result': 'SUCCESS'}


In [19]:
NEW_BMI323_CONFIGURATION = {
        "targetType": TargetType.I3C_DEVICE,
        "IBIRequest": TargetInterruptRequest.ACCEPT_IBI,
        "CRRequest": ControllerRoleRequest.REJECT_CRR,                  # Reject CRR
        "daaUseSETDASA": SetdasaConfiguration.DO_NOT_USE_SETDASA,
        "daaUseSETAASA": SetaasaConfiguration.DO_NOT_USE_SETAASA,
        "daaUseENTDAA": EntdaaConfiguration.USE_ENTDAA,
        "ibiTimestampEnable": IBiTimestamp.DISABLE_IBIT,
        "pendingReadCapability": PendingReadCapability.DISABLE_AUTOMATIC_READ
}

supernova.i3cControllerSetTargetDeviceConfiguration(getId(), BMI323_DYNAMIC_ADDRESS, NEW_BMI323_CONFIGURATION)

{'module': 1,
 'opcode': 0,
 'message': 'I3C SET TARGET DEVICE CONFIG request success'}

>> New message from SUPERNOVA:
{'id': 16, 'command': 'I3C CONTROLLER SET TARGET DEVICE CONFIG', 'result': 'SUCCESS'}


The line of code below showcase the error returned when the dynamic address is not part of the target devices table.

In [20]:
supernova.i3cControllerSetTargetDeviceConfiguration(getId(), 0x70, NEW_LSMD6V_CONFIGURATION)

{'module': 1,
 'opcode': 0,
 'message': 'I3C SET TARGET DEVICE CONFIG request success'}

>> New message from SUPERNOVA:
{'id': 17, 'command': 'I3C CONTROLLER SET TARGET DEVICE CONFIG', 'result': 'TARGET_ADDRESS_NOT_FOUND_IN_THE_TABLE'}


In [21]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 18, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 3, 'table': [{'static_address': 80, 'dynamic_address': 0, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I2C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 107, 'dynamic_address': 10, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_re

### 5. Set Frequencies

``Supernova.i3cControllerSetParameters(id, pushPullRate, i3cOpenDrainRate, i2cOpenDrainRate)``

This method allows the user to set the different frequencies.

  - `id: int`: A 2-byte integer that represents the transfer ID.
  - `pushPullRate: I3cPushPullTransferRate`: Push-Pull frequency.
  - `i3cOpenDrainRate: I3cOpenDrainTransferRate`: I3C frequency in Open Drain mode.
  - `i2cOpenDrainRate: I2cTransferRate`: I2C frequency.

In [22]:
supernova.i3cControllerSetParameters(getId(), I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ, I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ, I2cTransferRate._1MHz)

{'module': 1,
 'opcode': 0,
 'message': 'I3C CONTROLLER SET PARAMETERS request success'}

>> New message from SUPERNOVA:
{'id': 19, 'command': 'I3C CONTROLLER SET PARAMETERS', 'result': 'SUCCESS'}


### 6. I3C Private Transfers

#### 6.1. I2C Legacy over I3C

The Supernova, as stated in the MIPI I3C specification, supports backward compatibility with I2C. When an I2C device is connected to the bus,
the device must be added to the I3C Target Devices Table as explained above in section 3.3. Then the Supernova API allows to issue I2C Write,
I2C Read and I2C Read From (I2C Write + RS + I3C Read) transfers.

In [23]:
ID              = getId()
TARGET_ADDR     = 0x50
MODE            = TransferMode.I2C_MODE
REG_ADDR        = [0x00, 0x00]
DATA            = [33 for i in range(10)]

# API I3C Read
request_result = supernova.i3cControllerWrite(ID, TARGET_ADDR, MODE, REG_ADDR, DATA)

>> New message from SUPERNOVA:
{'id': 20, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 10}


In [24]:
ID              = getId()
TARGET_ADDR     = 0x50
MODE            = TransferMode.I2C_MODE
REG_ADDR        = [0x00,0x00]                                       # If this list is empty [], then only and I2C Read transfer is issued.
LEN             = 77

# API I3C Read
request_result = supernova.i3cControllerRead(ID, TARGET_ADDR, MODE, REG_ADDR, LEN)

>> New message from SUPERNOVA:
{'id': 21, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 77, 'payload': [33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]}


#### 6.2 Private Write/Read Transfers

#### 6.2.1 Private Read Transfers

In [25]:
TARGET_ADDR     = 0x0A
MODE            = TransferMode.I3C_SDR
REG_ADDR        = [0x00]
LEN             = 6

# API I3C Read
request_result = supernova.i3cControllerRead(getId(), TARGET_ADDR, MODE, REG_ADDR, LEN, startWith7E=False)

>> New message from SUPERNOVA:
{'id': 22, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 6, 'payload': [0, 0, 35, 0, 0, 0]}


#### 6.2.2 Private Write Transfers

In [26]:
TARGET_ADDR     = 0x0B
MODE            = TransferMode.I3C_SDR
REG_ADDR        = [0x05, 0x06]
DATA            = [i for i in range(10)]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cControllerWrite(getId(),TARGET_ADDR,MODE,REG_ADDR,DATA,startWith7E=False)

>> New message from SUPERNOVA:
{'id': 23, 'command': 'I3C CONTROLLER PRIVATE TRANSFER', 'result': 'SUCCESS', 'payload_length': 10}


#### 7 Common command code

##### 7.1 GETPID

In [27]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETPID(getId(),TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 24, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETPID', 'payload_length': 6, 'payload': [2, 8, 0, 112, 146, 11]}


##### 7.2 GETBCR

In [28]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETBCR(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 25, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETBCR', 'payload_length': 1, 'payload': [7]}


##### 7.3 GETDCR

In [29]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETDCR(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 26, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETDCR', 'payload_length': 1, 'payload': [68]}


##### 7.4 GETCAPS

In [30]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETCAPS(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 27, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETCAPS', 'payload_length': 4, 'payload': [0, 17, 24, 0]}


##### 7.5 GETMXDS

In [31]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETMXDS(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 28, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMXDS', 'payload_length': 2, 'payload': [8, 96]}


##### 7.6 GETMRL

In [32]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETMRL(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 29, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMRL', 'payload_length': 3, 'payload': [0, 16, 10]}


##### 7.7 Direct SETMRL

In [33]:
TARGET_ADDR = 0x0A
MAXIMUM_READ_LEN = 127
IBI_PAYLOAD_SIZE = 0x03

request_result = supernova.i3cDirectSETMRL(getId(), TARGET_ADDR, MAXIMUM_READ_LEN, IBI_PAYLOAD_SIZE)

>> New message from SUPERNOVA:
{'id': 30, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMRL', 'payload_length': 3}


##### 7.8 Braodcast SETMRL

In [34]:
MAXIMUM_READ_LEN = 0x0052
IBI_PAYLOAD_SIZE = 0x04

request_result = supernova.i3cBroadcastSETMRL(getId(), MAXIMUM_READ_LEN, IBI_PAYLOAD_SIZE)

>> New message from SUPERNOVA:
{'id': 31, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETMRL', 'payload_length': 4}


##### 7.9 GETMWL

In [35]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETMWL(getId(), TARGET_ADDR)

>> New message from SUPERNOVA:
{'id': 32, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMWL', 'payload_length': 2, 'payload': [0, 8]}


##### 7.10 Direct SETMWL

In [36]:
TARGET_ADDR = 0x0A
MAXIMUM_WRITE_LEN = 0X0102

request_result = supernova.i3cDirectSETMWL(getId(),
                                            TARGET_ADDR,
                                            MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{'id': 33, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETMWL', 'payload_length': 2}


##### 7.11 Broadcast SETMWL

In [37]:
MAXIMUM_WRITE_LEN = 0X0304

request_result = supernova.i3cBroadcastSETMWL(getId(), MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{'id': 34, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_SETMWL', 'payload_length': 3}


##### 7.12 Direct DISEC

In [38]:
request_result = supernova.i3cDirectDISEC(getId(), LSMD6V_DYNAMIC_ADDRESS, [ENEC.ENINT, ENEC.ENHJ])

>> New message from SUPERNOVA:
{'id': 35, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_DISEC', 'payload_length': 1}


##### 7.13 Broadcast ENEC

In [39]:
request_result = supernova.i3cBroadcastENEC(getId(), [DISEC.DISHJ])

>> New message from SUPERNOVA:
{'id': 36, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_ENEC', 'payload_length': 2}


In [40]:
NEW_BMI323_DYN_ADDR = 0x21

request_result = supernova.i3cSETNEWDA(getId(), BMI323_DYNAMIC_ADDRESS, NEW_BMI323_DYN_ADDR)

>> New message from SUPERNOVA:
{'id': 37, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_SETNEWDA', 'payload_length': 1}


In [42]:
request_result = supernova.i3cGETMWL(getId(), BMI323_DYNAMIC_ADDRESS)

>> New message from SUPERNOVA:
{'id': 39, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'I3C_NACK_ADDRESS', 'ccc': 'D_GETMWL', 'payload_length': 0, 'payload': []}


In [43]:
request_result = supernova.i3cGETMWL(getId(), NEW_BMI323_DYN_ADDR)

>> New message from SUPERNOVA:
{'id': 40, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_GETMWL', 'payload_length': 2, 'payload': [0, 0]}


In [44]:
request_result = supernova.i3cControllerGetTargetDevicesTable(getId())

>> New message from SUPERNOVA:
{'id': 41, 'command': 'I3C CONTROLLER GET TARGET DEVICES TABLE', 'result': 'SUCCESS', 'number_of_targets': 3, 'table': [{'static_address': 80, 'dynamic_address': 0, 'pid': [0, 0, 0, 0, 0, 0], 'bcr': 0, 'dcr': 0, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I2C_DEVICE', 'interrupt_request': 'REJECT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'DO_NOT_USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pending_read_capability': 'DISABLE_AUTOMATIC_READ'}}, {'static_address': 107, 'dynamic_address': 10, 'pid': [2, 8, 0, 112, 146, 11], 'bcr': 7, 'dcr': 68, 'mwl': 0, 'mrl': 0, 'max_ibi_payload_length': 0, 'configuration': {'target_type': 'I3C_DEVICE', 'interrupt_request': 'ACCEPT_IBI', 'controller_role_request': 'REJECT_CRR', 'setdasa': 'USE_SETDASA', 'setaasa': 'DO_NOT_USE_SETAASA', 'entdaa': 'DO_NOT_USE_ENTDAA', 'ibi_timestamp': 'DISABLE_IBIT', 'pend

### 8. I3C Target Reset

The Supernova as the I3C Controller provides support to the I3C Target Reset Pattern. The SUpernova class provides methods to support the RSTACT CCC and a separated method to trigger the I3C Target Reset Pattern into the bus.

##### 8.1 Direct RSTACT in read mode

In [45]:
TARGET_ADDR     = 0x0A

request_result = supernova.i3cDirectRSTACT(getId(), TARGET_ADDR, I3cTargetResetDefByte.RESET_I3C_PERIPHERAL, TransferDirection.READ)

>> New message from SUPERNOVA:
{'id': 42, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_RSTACT', 'payload_length': 1, 'payload': [0]}


##### 8.2 Direct RSTACT in write mode

In [46]:
TARGET_ADDR     = 0x0A

request_result = supernova.i3cDirectRSTACT(getId(), TARGET_ADDR, I3cTargetResetDefByte.RESET_I3C_PERIPHERAL, TransferDirection.WRITE)

>> New message from SUPERNOVA:
{'id': 43, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'D_RSTACT', 'payload_length': 2}


##### 8.3 Trigger the Target Reset Pattern

In [47]:
request_result = supernova.i3cControllerTriggerTargetResetPattern(getId())

>> New message from SUPERNOVA:
{'id': 44, 'command': 'I3C CONTROLLER TRIGGER PATTERN', 'result': 'SUCCESS', 'pattern': 'I3C_TARGET_RESET_PATTERN'}


##### 8.4 Reset all the whole Targets using Broadcast RSTACT CCC.

In [48]:
TARGET_ADDR     = 0x0A

request_result = supernova.i3cBroadcastRSTACT(getId(), I3cTargetResetDefByte.RESET_WHOLE_TARGET)

>> New message from SUPERNOVA:
{'id': 45, 'command': 'I3C CONTROLLER CCC TRANSFER', 'result': 'SUCCESS', 'ccc': 'B_RSTACT', 'payload_length': 2}


In [49]:
request_result = supernova.i3cControllerTriggerTargetResetPattern(getId())

>> New message from SUPERNOVA:
{'id': 46, 'command': 'I3C CONTROLLER TRIGGER PATTERN', 'result': 'SUCCESS', 'pattern': 'I3C_TARGET_RESET_PATTERN'}


### 9. I3C HDR Exit Pattern

The Supernova as the I3C Controller provides support to the I3C HDR Exit Pattern.

In [50]:
request_result = supernova.i3cControllerTriggerHdrExitPattern(getId())

>> New message from SUPERNOVA:
{'id': 47, 'command': 'I3C CONTROLLER TRIGGER PATTERN', 'result': 'SUCCESS', 'pattern': 'I3C_HDR_EXIT_PATTERN'}


### 10. Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [51]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}